# 7. Bandit Algorithms

TODO

## Hoeffding's inequality
> [...] provides an upper bound on the probability that the sum of random variables deviates from its expected value. 
>
> -- Wikipedia

Let $X_1,\dots,X_m$ be i.i.d. random variables taking values in $[0, 1]$.

The real mean $\mu = \mathbb{E}[X]$ is unknown.

We have an empirical estimate based on our $m$ trials:

\begin{equation}
    \hat{\mu}_m = \frac{1}{m} \sum_{i = 1}^{m} X_i
\end{equation}

Then we have:

\begin{equation}
    P\left(\left|\mu - \hat{\mu}_m\right| \ge b\right) \le 2 \exp\left(-2b^2m \right) = \delta_m
\end{equation}

That is, the probability of our operation being more than $b$ off the real value is smaller than a computable threshold.

We just want an upper bound. So we set it to, say, $\delta_m$, and then compute the corresponding $b$, as a function of $\delta_m$ and $m$.

For a fixed upper probability bound, we fix $\delta_m$ and get $b = \sqrt{\frac{1}{2m} \log{\frac{2}{\delta_m}}}$.

Now we also want to set an upper bound on the probability not just for a single $m$, but for all $m$. Want upper bound for $E_m = P(|\mu - \hat{\mu}_m| \ge b)$, i.e. a lower bound for $P(|\mu - \hat{\mu}_t| \ge b, \> \forall t)$.

So we get:

\begin{equation}
\begin{aligned}
    P(|\mu - \hat{\mu}_t| \ge b, \> \forall t) & = 1 - P(E_1 \cup E_2 \cup \dots) \\
    & \ge 1 - \sum_{t=1}^{\infty} P(E_t) \\
    & \ge 1 - \sum_{t=1}^{\infty} \delta_t \\
    & \ge 1 - \delta \> \text{with} \> \delta < \sum_{t=1}^{\infty}\delta_t
\end{aligned}
\end{equation}

2nd row smaller since the sum we're subtracting is bigger (longer than the prev.).
3rd row smaller because all deltas are larger than their corresponding Es, as defined by Hoeffding's inequality itself.

We therefore want a sum of $d_t$ which is bounded. Setting $\delta_t = \frac{c}{t^2}$ works well, since the correspoinding sum converges, so the upper bound $\delta$ exists and is finite.

We now have a good heuristic: at any time step $t$, our upper bound should be $\delta_t = \frac{c}{t^2}$.

Recall that we want to express $b$ as a function of $\delta_t$, since $b$ is *the value which actually defines our upper confidence bound*.

(Note that this probability shrinks quadratically over time, so it keeps getting tighter and tighter for all arms, as we keep playing.)

All we need to do now is shove our $\delta_t$ in the formula for $b$, and we get (setting $c := 2$):

\begin{aligned}
\operatorname{UCB}(i) 
& = \hat{\mu}_i + \sqrt{\frac{1}{n_i} \ln (2  \frac{t^2}{2}) } \\
& = \hat{\mu}_i + \sqrt{\frac{\ln{t^2}}{n_i}} \\
& = \hat{\mu}_i + \sqrt{\frac{2 \ln{t}}{n_i}}
\end{aligned}

We can plug this formula right into a program! See `bonus/tutorial-bandits.ipynb` for an implementation.

TODO: there seems to be a "2" missing somewhere. Low priority.